# 层级索引创建多维df

In [14]:
import pandas as pd
import numpy as np

# 创建 3x3x2 的三维数据
data = np.random.randn(3, 3, 2)  # 3x3x2 数据，18个元素

# 创建 MultiIndex （确保索引数量与数据数量一致）
arrays = [
    ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],  # Dim1
    ['1', '2', '3', '1', '2', '3', '1', '2', '3', '1', '2', '3', '1', '2', '3', '1', '2', '3'],  # Dim2
]

# 创建 MultiIndex
index = pd.MultiIndex.from_arrays(arrays, names=('Dim1', 'Dim2'))

# 创建 DataFrame，确保数据与索引数量匹配
df = pd.DataFrame(data.flatten(), index=index, columns=['Value'])
df = df.sort_index()
# print(df)
df_a_2 = df.loc['A','2']
print(df_a_2)
# print(df_a.loc['2'])

              Value
Dim1 Dim2          
A    2    -0.150791
     2     1.515344


# 数据链接 Merge

In [ ]:
"""
参数1： 链接方式 how ：
        1.how = 'ineer' 即内连接，会选择在两个df中都存在的key
        2.how = 'outer' 外链接，会返回两个df中的所有行，如有一侧没匹配，就会填入nan
        3.how = 'left'  左连接，保留左侧的所有行，右侧如果没有，就填nan
        4.how = 'right' 右连接，保留右侧行，左侧如果没有就nan
        
参数2   基于哪个列 on:
        1. on = 'key' 搭配inner就是 两侧都有key的行
        2. left_on = 'key' 搭配inner就是等于 left，on
        3. right_on = 'key' 同理
"""

In [16]:
import pandas as pd
import numpy as np
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
'data2' : np.random.randint(0,10,3)})
print(df_obj1)
print(df_obj2)
print(df_obj1.merge(df_obj2))

  key  data1
0   b      1
1   b      1
2   a      7
3   c      8
4   a      8
5   a      4
6   b      5
  key  data2
0   a      8
1   b      6
2   d      2
  key  data1  data2
0   b      1      6
1   b      1      6
2   a      7      8
3   a      8      8
4   a      4      8
5   b      5      6


In [17]:
# on 显示指定“外键”
print(pd.merge(df_obj1, df_obj2, on='key'))

  key  data1  data2
0   b      1      6
1   b      1      6
2   a      7      8
3   a      8      8
4   a      4      8
5   b      5      6


In [30]:
# left_on，right_on 分别指定左侧数据和右侧数据的“外键”
# 更改列名
df_obj1 = df_obj1.rename(columns={'key':'key1'})
df_obj2 = df_obj2.rename(columns={'key':'key2'})
merge_df = pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2')
print(merge_df)

  key1  data1 key2  data2
0    b      1    b      6
1    b      1    b      6
2    a      7    a      8
3    a      8    a      8
4    a      4    a      8
5    b      5    b      6


如果已经得到了 key1 data1 key2 data2这样的上述df，怎样回到key data1 data2的形式

In [32]:
merge_df = merge_df.drop(columns=['key1'])
merge_df = merge_df.rename(columns = {'key2':'key'})
print(merge_df)

KeyError: "['key1'] not found in axis"

In [37]:
# “外连接”
print(pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='outer'))

  key1  data1 key2  data2
0    a    7.0    a    8.0
1    a    8.0    a    8.0
2    a    4.0    a    8.0
3    b    1.0    b    6.0
4    b    1.0    b    6.0
5    b    5.0    b    6.0
6    c    8.0  NaN    NaN
7  NaN    NaN    d    2.0


In [38]:
# 左连接
print(pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='left'))

  key1  data1 key2  data2
0    b      1    b    6.0
1    b      1    b    6.0
2    a      7    a    8.0
3    c      8  NaN    NaN
4    a      8    a    8.0
5    a      4    a    8.0
6    b      5    b    6.0


In [39]:
# 右连接
print(pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='right'))

  key1  data1 key2  data2
0    a    7.0    a      8
1    a    8.0    a      8
2    a    4.0    a      8
3    b    1.0    b      6
4    b    1.0    b      6
5    b    5.0    b      6
6  NaN    NaN    d      2


In [40]:
# 处理重复列名

"""
这里的重复列名指的是除了要用于合并的key之外的重复列名，然后分别添上_left 和_right的后缀
而且这个suffix厉害在他可以把所有相同的列都加上后缀，
"""
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
'data' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
'data' : np.random.randint(0,10,3)})
print(pd.merge(df_obj1, df_obj2, on='key', suffixes=('_left', '_right')))

  key  data_left  data_right
0   b          4           6
1   b          5           6
2   a          4           1
3   a          3           1
4   a          1           1
5   b          4           6


In [42]:
# 按索引连接
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'data2' : np.random.randint(0,10,3)}, index=['a', 'b', 'd'])
print(df_obj1)
print(df_obj2)
print(pd.merge(df_obj1, df_obj2, left_on='key', right_index=True))

  key  data1
0   b      3
1   b      5
2   a      0
3   c      0
4   a      9
5   a      3
6   b      6
   data2
a      5
b      1
d      5
  key  data1  data2
0   b      3      1
1   b      5      1
2   a      0      5
4   a      9      5
5   a      3      5
6   b      6      1


# 重复数据处理

In [51]:

import numpy as np
import pandas as pd

df_obj = pd.DataFrame({'data1' : ['a'] * 4 + ['b'] * 4,
'data2' : np.random.randint(0, 4, 8)})
print(df_obj)
print('\n')
print(df_obj.duplicated())
print('\n')

#丢掉重复 drop_duplicates,默认全部列，也可以选择指定列
print(df_obj.drop_duplicates())
print(df_obj.drop_duplicates('data2'))



  data1  data2
0     a      3
1     a      2
2     a      0
3     a      2
4     b      3
5     b      3
6     b      3
7     b      3


0    False
1    False
2    False
3     True
4    False
5     True
6     True
7     True
dtype: bool


  data1  data2
0     a      3
1     a      2
2     a      0
4     b      3
  data1  data2
0     a      3
1     a      2
2     a      0


In [52]:
#. 根据 map 传入的函数对每行或每列进行转换
ser_obj = pd.Series(np.random.randint(0,10,10))
print(ser_obj)
print(ser_obj.map(lambda x : x ** 2))

0    3
1    5
2    7
3    3
4    8
5    0
6    5
7    3
8    3
9    8
dtype: int64
0     9
1    25
2    49
3     9
4    64
5     0
6    25
7     9
8     9
9    64
dtype: int64


数据替换

In [53]:
# 单个值替换单个值
print(ser_obj.replace(1, -100))
# 多个值替换一个值
print(ser_obj.replace([6,8], -100))
# 多个值替换多个值
print(ser_obj.replace([4, 7], [-100, -200]))

0    3
1    5
2    7
3    3
4    8
5    0
6    5
7    3
8    3
9    8
dtype: int64
0      3
1      5
2      7
3      3
4   -100
5      0
6      5
7      3
8      3
9   -100
dtype: int64
0      3
1      5
2   -200
3      3
4      8
5      0
6      5
7      3
8      3
9      8
dtype: int64
